### We have the library inventory and checkouts data from Seattle Public Library. Our objective is to predict which books will be checked out next month. Lets load up the datasets.

In [1]:
### Initiating Spark Context
sc

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
6,application_1579135619846_0007,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-6>

In [194]:
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import desc, col, udf, max as max_, lit
from datetime import datetime
from pyspark.sql.types import DateType, IntegerType

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
ss = SparkSession.builder.getOrCreate()
sqlcontext = SQLContext(sc)
inventory_df = sqlcontext.read.parquet('s3://intersession-distcomp/Inventory')
inventory_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- BibNum: integer (nullable = true)
 |-- PublicationYear: integer (nullable = true)
 |-- Author: string (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- ItemType: string (nullable = true)
 |-- ItemCollection: string (nullable = true)
 |-- ItemLocation: string (nullable = true)
 |-- ItemCount: integer (nullable = true)
 |-- ReportDate: date (nullable = true)

In [4]:
checkouts_df = sqlcontext.read.parquet('s3://intersession-distcomp/Checkouts')
checkouts_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- BibNumber: integer (nullable = true)
 |-- CheckoutDateTime: date (nullable = true)

In [5]:
inventory_df.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+---------------+--------------------+--------------------+--------+--------------+------------+---------+----------+
| BibNum|PublicationYear|              Author|           Publisher|ItemType|ItemCollection|ItemLocation|ItemCount|ReportDate|
+-------+---------------+--------------------+--------------------+--------+--------------+------------+---------+----------+
|3068628|           2015|                null|            Dey St.,|    acbk|          canf|         cen|        1|2019-02-01|
|3118549|           2015|Wilco (Musical gr...| DBPM Records/Anti-,|    accd|          nacd|         nga|        3|2019-02-01|
|3162819|           2016| Bosch, Pseudonymous|Blackstone Audio ...|    jccd|        ncbocd|         wal|        1|2019-02-01|
|1924473|           1999|   Geeslin, Campbell|Atheneum Books fo...|    jcbk|         ccpic|         cen|        1|2019-02-01|
|1815199|           1996|United States. Na...|National Transpor...|    acbk|          canf|         cen|        2|2019

In [6]:
checkouts_df.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+----------------+
|BibNumber|CheckoutDateTime|
+---------+----------------+
|  3027450|      2015-05-27|
|  2969577|      2015-04-18|
|  2932682|      2014-07-25|
|  2730431|      2014-08-25|
|  2567182|      2015-07-13|
|  2721456|      2015-06-15|
|  2802494|      2015-06-14|
|  2947048|      2014-11-13|
|  2954535|      2014-11-24|
|  3010547|      2015-03-13|
+---------+----------------+
only showing top 10 rows

### To get the Train and Test datasets, we need to subset Inventory as it is incrementally updated for books every month. Since we have checkout data till December 2019, we will subset our train dataset from November Inventory data and train dataset from December Inventory Data.

In [8]:
subset = inventory_df.filter(inventory_df['ReportDate'] <= '2019-11-01').groupby('BibNum').agg(max_("ReportDate"))
subset = subset.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
subset_test = inventory_df.filter(inventory_df['ReportDate'] <= '2019-12-01').groupby('BibNum').agg(max_("ReportDate"))
subset_test = subset_test.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
print(subset.count(), subset_test.count()) ### 785267, 787918

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

785267 787918

In [144]:
df_train = inventory_df.join(subset.withColumnRenamed('max(ReportDate)','ReportDate'), on=['BibNum','ReportDate'], how='inner')
df_test = inventory_df.join(subset_test.withColumnRenamed('max(ReportDate)','ReportDate'), on=['BibNum','ReportDate'], how='inner')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [145]:
df_train = df_train.cache()
df_test = df_test.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [146]:
print(df_train.count()) ###  1549043
print(df_test.count()) ###  1551509

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1549043
1551509

In [147]:
df_train = df_train.drop('Title','ISBN','Subjects')
df_test = df_test.drop('Title','ISBN','Subjects')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [148]:
df_train.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- BibNum: integer (nullable = true)
 |-- ReportDate: date (nullable = true)
 |-- PublicationYear: integer (nullable = true)
 |-- Author: string (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- ItemType: string (nullable = true)
 |-- ItemCollection: string (nullable = true)
 |-- ItemLocation: string (nullable = true)
 |-- ItemCount: integer (nullable = true)

In [149]:
df_train.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------+---------------+--------------------+--------------------+--------+--------------+------------+---------+
| BibNum|ReportDate|PublicationYear|              Author|           Publisher|ItemType|ItemCollection|ItemLocation|ItemCount|
+-------+----------+---------------+--------------------+--------------------+--------+--------------+------------+---------+
|2676669|2019-02-01|           2010|Stanton, Steve, 1...|          ECW Press,|    acbk|         nanew|         dth|        1|
|2944868|2019-02-01|           2006|       Edgers, Geoff|   Grosset & Dunlap,|    jcbk|          ncnf|         net|        1|
| 379359|2019-02-01|           1984|Vishnevska︠i︡a, G...|Harcourt Brace Jo...|    acbk|          canf|         cen|        1|
|2243510|2019-02-01|           2004|McKissack, Pat, 1...|         Scholastic,|    jcbk|         ccser|         cen|        1|
| 331198|2019-02-01|           1963|Trenker, Luis, 18...|       Fackelverlag,|    acbk|          caln|         cen|   

In [150]:
df_test.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------+---------------+--------------------+--------------------+--------+--------------+------------+---------+
| BibNum|ReportDate|PublicationYear|              Author|           Publisher|ItemType|ItemCollection|ItemLocation|ItemCount|
+-------+----------+---------------+--------------------+--------------------+--------+--------------+------------+---------+
|2676669|2019-02-01|           2010|Stanton, Steve, 1...|          ECW Press,|    acbk|         nanew|         dth|        1|
|2944868|2019-02-01|           2006|       Edgers, Geoff|   Grosset & Dunlap,|    jcbk|          ncnf|         net|        1|
| 379359|2019-02-01|           1984|Vishnevska︠i︡a, G...|Harcourt Brace Jo...|    acbk|          canf|         cen|        1|
|2243510|2019-02-01|           2004|McKissack, Pat, 1...|         Scholastic,|    jcbk|         ccser|         cen|        1|
| 331198|2019-02-01|           1963|Trenker, Luis, 18...|       Fackelverlag,|    acbk|          caln|         cen|   

### Also the books are not unique at BibNum level, hence we need to aggreagate the rest of the cateogorical variables

In [151]:
df_train = df_train.groupBy('BibNum').agg({'ReportDate':'max', 'Author':'max', 'PublicationYear':'max', 'Publisher':'max', 
                               'ItemType':'max', 'ItemCollection':'max', 'ItemCount':'sum', 'ItemLocation':'max'})
df_test = df_test.groupBy('BibNum').agg({'ReportDate':'max', 'Author':'max', 'PublicationYear':'max', 'Publisher':'max', 
                               'ItemType':'max', 'ItemCollection':'max', 'ItemCount':'sum', 'ItemLocation':'max'})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [153]:
df_train = df_train.select(col("BibNum"), col("max(ReportDate)").alias("ReportDate"), col("sum(ItemCount)").alias("ItemCount"), 
               col("max(ItemCollection)").alias("ItemCollection"), col("max(PublicationYear)").alias("PublicationYear"),
               col("max(Author)").alias("Author"), col("max(ItemType)").alias("ItemType"), col("max(Publisher)").alias("Publisher")
                          , col('max(ItemLocation)').alias('ItemLocation'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [154]:
df_test = df_test.select(col("BibNum"), col("max(ReportDate)").alias("ReportDate"), col("sum(ItemCount)").alias("ItemCount"), 
               col("max(ItemCollection)").alias("ItemCollection"), col("max(PublicationYear)").alias("PublicationYear"),
               col("max(Author)").alias("Author"), col("max(ItemType)").alias("ItemType"), col("max(Publisher)").alias("Publisher")
                        , col('max(ItemLocation)').alias('ItemLocation'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [155]:
df_train.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+----------+---------+--------------+---------------+--------------------+--------+--------------------+------------+
|BibNum|ReportDate|ItemCount|ItemCollection|PublicationYear|              Author|ItemType|           Publisher|ItemLocation|
+------+----------+---------+--------------+---------------+--------------------+--------+--------------------+------------+
|  1238|2019-11-01|        1|          cs7r|           1967|Simon, André Loui...|    arbk|        McGraw-Hill,|         cen|
|  1342|2019-11-01|        1|          cs7r|           1967|Great Britain. Co...|    arbk|           H.M.S.O.,|         cen|
|  1580|2019-11-01|        1|         casea|           1968|Sissons, John How...|    arbk|McClelland and St...|         cen|
|  1591|2019-11-01|        1|          canf|           1965|Smith, George Was...|    acbk|[publisher not id...|         cen|
|  2866|2018-08-01|        1|          canf|           1969|Thomason, John W....|    acbk|University of Tex...|         cen|


## Feature Engineering

### 1. PublicationYear - fill na's with mean

In [157]:
from pyspark.sql.functions import mean
mean_year = int(df_train.select([mean('PublicationYear')]).first()[0])
df_train = df_train.fillna(mean_year, subset=['PublicationYear'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [158]:
df_train.groupBy('PublicationYear').count().sort('count', ascending=False).show(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+-----+
|PublicationYear|count|
+---------------+-----+
|           1993|49222|
|           2015|28464|
|           2016|28312|
|           2013|28247|
|           2014|27702|
|           2017|27519|
|           2012|27130|
|           2010|26908|
|           2018|25576|
|           2011|25565|
|           2009|24627|
|           2008|23556|
|           2007|19032|
|           2019|17718|
|           2006|16593|
|           2005|14626|
|           2004|14140|
|           2002|13935|
|           2003|13860|
|           2000|13450|
|           2001|13077|
|           1999|12904|
|           1998|12186|
|           1997|10928|
|           1996| 9959|
|           1995| 9860|
|           1994| 9617|
|           1992| 8707|
|           1990| 8047|
|           1991| 7774|
|           1989| 7447|
|           1988| 5757|
|           1987| 5506|
|           1986| 4967|
|           1969| 4942|
|           1968| 4689|
|           1985| 4678|
|           1976| 4606|
|           1984

In [159]:
df_test = df_test.fillna(mean_year, subset=['PublicationYear'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [160]:
df_test.groupBy('PublicationYear').count().sort('count', ascending=False).show(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+-----+
|PublicationYear|count|
+---------------+-----+
|           1993|49244|
|           2015|28488|
|           2016|28346|
|           2013|28271|
|           2014|27726|
|           2017|27583|
|           2012|27152|
|           2010|26928|
|           2018|25706|
|           2011|25581|
|           2009|24640|
|           2008|23575|
|           2019|19645|
|           2007|19045|
|           2006|16598|
|           2005|14646|
|           2004|14154|
|           2002|13945|
|           2003|13871|
|           2000|13461|
|           2001|13084|
|           1999|12909|
|           1998|12191|
|           1997|10936|
|           1996| 9962|
|           1995| 9867|
|           1994| 9622|
|           1992| 8707|
|           1990| 8050|
|           1991| 7781|
|           1989| 7452|
|           1988| 5759|
|           1987| 5509|
|           1986| 4968|
|           1969| 4942|
|           1968| 4690|
|           1985| 4681|
|           1976| 4609|
|           1984

### 2. Book counts for variable 'Author'

In [131]:
df_train.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

785267

In [161]:
auth = df_train.groupBy('Author').count()
df_train = df_train.join(auth,on='Author',how='left').drop('Author').withColumnRenamed('count','author_book_counts')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [162]:
df_train.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------+---------+--------------+---------------+--------+--------------------+------------+------------------+
| BibNum|ReportDate|ItemCount|ItemCollection|PublicationYear|ItemType|           Publisher|ItemLocation|author_book_counts|
+-------+----------+---------+--------------+---------------+--------+--------------------+------------+------------------+
|3123964|2019-11-01|        4|         nafic|           2016|    acbk|        Grove Press,|         glk|                 5|
|2686764|2019-11-01|        1|         cafic|           2011|    acbk|        Grove Press,|         cen|                 5|
|3409841|2019-11-01|        6|         nanew|           2019|    acbk|Black Cat, an imp...|         swt|                 5|
|3204018|2019-11-01|        6|         nafic|           2005|    acbk|          Black Cat,|         net|                 5|
|3204074|2019-11-01|        6|         nafic|           2006|    acbk|Black Cat ; Distr...|         swt|                 5|
|2749241

In [163]:
mean_book_cnts = int(df_train.select([mean('author_book_counts')]).first()[0])
df_train = df_train.fillna(mean_book_cnts, subset=['author_book_counts'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [164]:
df_train.where(df_train.author_book_counts.isNull()).show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+----------+---------+--------------+---------------+--------+---------+------------+------------------+
|BibNum|ReportDate|ItemCount|ItemCollection|PublicationYear|ItemType|Publisher|ItemLocation|author_book_counts|
+------+----------+---------+--------------+---------------+--------+---------+------------+------------------+
+------+----------+---------+--------------+---------------+--------+---------+------------+------------------+

In [165]:
df_test = df_test.join(auth,on='Author',how='left').drop('Author').withColumnRenamed('count','author_book_counts')
df_test = df_test.fillna(mean_book_cnts, subset=['author_book_counts'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [166]:
df_test.where(df_test.author_book_counts.isNull()).show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+----------+---------+--------------+---------------+--------+---------+------------+------------------+
|BibNum|ReportDate|ItemCount|ItemCollection|PublicationYear|ItemType|Publisher|ItemLocation|author_book_counts|
+------+----------+---------+--------------+---------------+--------+---------+------------+------------------+
+------+----------+---------+--------------+---------------+--------+---------+------------+------------------+

### Publisher Count

In [168]:
pub = df_train.groupBy('Publisher').count()
df_train = df_train.join(pub,on='Publisher',how='left').drop('Publisher').withColumnRenamed('count','publisher_book_counts')
df_train.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------+---------+--------------+---------------+--------+------------+------------------+---------------------+
| BibNum|ReportDate|ItemCount|ItemCollection|PublicationYear|ItemType|ItemLocation|author_book_counts|publisher_book_counts|
+-------+----------+---------+--------------+---------------+--------+------------+------------------+---------------------+
|2733621|2019-11-01|        1|          canf|           2011|    acbk|         cen|                60|                    1|
| 727488|2019-11-01|        1|         casea|           1977|    arbk|         cen|                60|                    1|
| 746183|2019-11-01|        8|          cs9g|           1875|    arbk|         cen|                 1|                    1|
| 742931|2019-11-01|        1|          cs6r|           1891|    arbk|         cen|                 1|                    1|
| 385595|2019-11-01|        1|         caref|           1984|    arbk|         cen|                 1|                    1|


In [170]:
mean_pub_book_cnts = int(df_train.select([mean('publisher_book_counts')]).first()[0])
df_train = df_train.fillna(mean_pub_book_cnts, subset=['publisher_book_counts'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [171]:
df_train.where(df_train.publisher_book_counts.isNull()).show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+----------+---------+--------------+---------------+--------+------------+------------------+---------------------+
|BibNum|ReportDate|ItemCount|ItemCollection|PublicationYear|ItemType|ItemLocation|author_book_counts|publisher_book_counts|
+------+----------+---------+--------------+---------------+--------+------------+------------------+---------------------+
+------+----------+---------+--------------+---------------+--------+------------+------------------+---------------------+

In [172]:
df_test = df_test.join(pub,on='Publisher',how='left').drop('Publisher').withColumnRenamed('count','publisher_book_counts')
df_test = df_test.fillna(mean_pub_book_cnts, subset=['publisher_book_counts'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [173]:
df_train.where(df_train.publisher_book_counts.isNull()).show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+----------+---------+--------------+---------------+--------+------------+------------------+---------------------+
|BibNum|ReportDate|ItemCount|ItemCollection|PublicationYear|ItemType|ItemLocation|author_book_counts|publisher_book_counts|
+------+----------+---------+--------------+---------------+--------+------------+------------------+---------------------+
+------+----------+---------+--------------+---------------+--------+------------+------------------+---------------------+

In [174]:
df_train.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------+---------+--------------+---------------+--------+------------+------------------+---------------------+
| BibNum|ReportDate|ItemCount|ItemCollection|PublicationYear|ItemType|ItemLocation|author_book_counts|publisher_book_counts|
+-------+----------+---------+--------------+---------------+--------+------------+------------------+---------------------+
|2733621|2019-11-01|        1|          canf|           2011|    acbk|         cen|                60|                    1|
| 727488|2019-11-01|        1|         casea|           1977|    arbk|         cen|                60|                    1|
| 746183|2019-11-01|        8|          cs9g|           1875|    arbk|         cen|                 1|                    1|
| 742931|2019-11-01|        1|          cs6r|           1891|    arbk|         cen|                 1|                    1|
| 385595|2019-11-01|        1|         caref|           1984|    arbk|         cen|                 1|                    1|


### Now we add labels to our train and test dataset. Label for train dataset will come from checkouts from Nov and for test, they come from December.

In [182]:
checkout_df = sqlcontext.read.parquet('s3://intersession-distcomp/Checkouts')
checkout_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- BibNumber: integer (nullable = true)
 |-- CheckoutDateTime: date (nullable = true)

In [183]:
checkout_df = checkout_df.withColumnRenamed('BibNumber','BibNum').withColumnRenamed('CheckoutDateTime','date')
checkout_df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------+
| BibNum|      date|
+-------+----------+
|3027450|2015-05-27|
|2969577|2015-04-18|
|2932682|2014-07-25|
|2730431|2014-08-25|
|2567182|2015-07-13|
+-------+----------+
only showing top 5 rows

In [185]:
checkouts_nov = checkout_df.filter((checkout_df['date'] >= '2019-11-01') & (checkout_df['date'] <= '2019-11-30'))
checkouts_nov = checkouts_nov.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [186]:
### large perc of books are being checked out multiple times
checkouts_nov.select('BibNum').distinct().count() / checkouts_nov.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.29027123295416246

In [195]:
checkouts_nov_dist = checkouts_nov.select('BibNum').distinct().withColumn("Test",lit(1))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [196]:
checkouts_nov_dist.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----+
| BibNum|Test|
+-------+----+
|3374689|   1|
|2767269|   1|
|3078117|   1|
|3469318|   1|
|3375992|   1|
+-------+----+
only showing top 5 rows

In [207]:
checkouts_dec = checkout_df.filter((checkout_df['date'] >= '2019-12-01') & (checkout_df['date'] <= '2019-12-31'))
checkouts_dec = checkouts_dec.cache()
checkouts_dec_dist = checkouts_dec.select('BibNum').distinct().withColumn("Test",lit(1))
checkouts_dec_dist.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----+
| BibNum|Test|
+-------+----+
|3400790|   1|
|3219719|   1|
|3503833|   1|
|3078117|   1|
|2475246|   1|
+-------+----+
only showing top 5 rows

### Creating Labels on test and train datasets

In [204]:
df_train = df_train.join(checkouts_nov_dist, on='BibNum', how='left').withColumnRenamed('Test','label')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [205]:
df_train = df_train.fillna(0, subset=['label'])
df_train.groupBy('label').count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+------+
|label| count|
+-----+------+
|    1|130988|
|    0|654279|
+-----+------+

In [208]:
df_test = df_test.join(checkouts_dec_dist, on='BibNum', how='left').withColumnRenamed('Test','label')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [209]:
df_test = df_test.fillna(0, subset=['label'])
df_test.groupBy('label').count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+------+
|label| count|
+-----+------+
|    1|126271|
|    0|661647|
+-----+------+

### Variables like Checkouts last 1 month etc should have a good predictive power. We will create a host of them for our model, creating just one for now.

In [215]:
checkout_oct = checkout_df.filter((checkout_df['date'] >= '2019-10-01') & (checkout_df['date'] <= '2019-10-31'))\
    .groupBy('BibNum').count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [216]:
checkout_oct = checkout_oct.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [217]:
checkout_oct.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

135568

In [218]:
df_train = df_train.join(checkout_oct, on='BibNum', how='left').withColumnRenamed('count','checkouts_l1m')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [219]:
df_train = df_train.fillna(0, subset=['checkouts_l1m'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [226]:
checkouts_nov = checkout_df.filter((checkout_df['date'] >= '2019-11-01') & (checkout_df['date'] <= '2019-11-30'))
checkouts_nov = checkouts_nov.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [227]:
checkouts_nov = checkouts_nov.groupBy('BibNum').count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [228]:
df_test = df_test.join(checkouts_nov, on='BibNum', how='left').withColumnRenamed('count','checkouts_l1m')
df_test = df_test.fillna(0, subset=['checkouts_l1m'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [229]:
df_test.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------+---------+--------------+---------------+--------+------------+------------------+---------------------+-----+-------------+
| BibNum|ReportDate|ItemCount|ItemCollection|PublicationYear|ItemType|ItemLocation|author_book_counts|publisher_book_counts|label|checkouts_l1m|
+-------+----------+---------+--------------+---------------+--------+------------+------------------+---------------------+-----+-------------+
|2733621|2019-12-01|        1|          canf|           2011|    acbk|         cen|                60|                    1|    0|            0|
| 727488|2019-12-01|        1|         casea|           1977|    arbk|         cen|                60|                    1|    0|            0|
| 746183|2019-12-01|        8|          cs9g|           1875|    arbk|         cen|                 1|                    1|    0|            0|
| 742931|2019-12-01|        1|          cs6r|           1891|    arbk|         cen|                 1|                    1|    0|

In [230]:
df_test.groupBy('checkouts_l1m').count().show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+------+
|checkouts_l1m| count|
+-------------+------+
|           26|    94|
|           29|    63|
|           65|     7|
|           19|   218|
|           54|    10|
|            0|654860|
|          348|     1|
|          167|     1|
|          155|     1|
|          113|     1|
+-------------+------+
only showing top 10 rows

#### Final Train and Test Dataframes

In [232]:
df_train = df_train.drop('ReportDate')
df_train.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+---------+--------------+---------------+--------+------------+------------------+---------------------+-----+-------------+
| BibNum|ItemCount|ItemCollection|PublicationYear|ItemType|ItemLocation|author_book_counts|publisher_book_counts|label|checkouts_l1m|
+-------+---------+--------------+---------------+--------+------------+------------------+---------------------+-----+-------------+
|2733621|        1|          canf|           2011|    acbk|         cen|                60|                    1|    0|            0|
| 727488|        1|         casea|           1977|    arbk|         cen|                60|                    1|    0|            0|
| 746183|        8|          cs9g|           1875|    arbk|         cen|                 1|                    1|    0|            0|
| 742931|        1|          cs6r|           1891|    arbk|         cen|                 1|                    1|    0|            0|
| 385595|        1|         caref|           1984|    arbk|   

In [233]:
df_test = df_test.drop('ReportDate')
df_test.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+---------+--------------+---------------+--------+------------+------------------+---------------------+-----+-------------+
| BibNum|ItemCount|ItemCollection|PublicationYear|ItemType|ItemLocation|author_book_counts|publisher_book_counts|label|checkouts_l1m|
+-------+---------+--------------+---------------+--------+------------+------------------+---------------------+-----+-------------+
|2733621|        1|          canf|           2011|    acbk|         cen|                60|                    1|    0|            0|
| 727488|        1|         casea|           1977|    arbk|         cen|                60|                    1|    0|            0|
| 746183|        8|          cs9g|           1875|    arbk|         cen|                 1|                    1|    0|            0|
| 742931|        1|          cs6r|           1891|    arbk|         cen|                 1|                    1|    0|            0|
| 385595|        1|         caref|           1984|    arbk|   

### Exporting to HDFS

In [234]:
df_train.write.option("path","/train").saveAsTable('train')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [235]:
df_test.write.option("path","/test").saveAsTable('test')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Reading the file back from HDFS

In [237]:
ss.sql("select * from parquet.`/train`").show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+---------+--------------+---------------+--------+------------+------------------+---------------------+-----+-------------+
| BibNum|ItemCount|ItemCollection|PublicationYear|ItemType|ItemLocation|author_book_counts|publisher_book_counts|label|checkouts_l1m|
+-------+---------+--------------+---------------+--------+------------+------------------+---------------------+-----+-------------+
| 193435|        1|         caref|           1969|    arbk|         cen|                 1|                    1|    0|            0|
|3035345|        1|          canf|           2014|    acbk|         cen|                 2|                    1|    0|            0|
|3119038|        4|          nanf|           2015|    acbk|         net|                 3|                    7|    0|            0|
|3038071|        3|          nanf|           2014|    acbk|         swt|                 3|                    7|    0|            0|
|3136759|        1|          canf|           2015|    acbk|   

In [238]:
ss.sql("select * from parquet.`/test`").show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+---------+--------------+---------------+--------+------------+------------------+---------------------+-----+-------------+
| BibNum|ItemCount|ItemCollection|PublicationYear|ItemType|ItemLocation|author_book_counts|publisher_book_counts|label|checkouts_l1m|
+-------+---------+--------------+---------------+--------+------------+------------------+---------------------+-----+-------------+
| 193435|        1|         caref|           1969|    arbk|         cen|                 1|                    1|    0|            0|
|3035345|        1|          canf|           2014|    acbk|         cen|                 2|                    1|    0|            0|
|3154601|        4|          nanf|           2015|    acbk|         net|                 1|                    7|    1|            0|
|3136759|        1|          canf|           2015|    acbk|         cen|                 1|                    7|    0|            0|
|3038071|        3|          nanf|           2014|    acbk|   

### Saving Train and Test datasets on s3 bucket

In [239]:
df_train.write.parquet('s3://intersession-distcomp/train_df')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [240]:
df_test.write.parquet('s3://intersession-distcomp/test_df')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Reading Test from s3 bucket

In [241]:
ss.read.parquet('s3://intersession-distcomp/test_df').show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+---------+--------------+---------------+--------+------------+------------------+---------------------+-----+-------------+
| BibNum|ItemCount|ItemCollection|PublicationYear|ItemType|ItemLocation|author_book_counts|publisher_book_counts|label|checkouts_l1m|
+-------+---------+--------------+---------------+--------+------------+------------------+---------------------+-----+-------------+
| 566774|        1|          caln|           1993|   arper|         cen|                60|                    1|    0|            0|
|2671059|        1|          cacd|           1994|    accd|         cen|                 5|                    2|    0|            0|
|2446970|        1|          nacd|           1993|    accd|         gwd|                 3|                    2|    0|            0|
|2872844|        5|       nadvdnf|           2012|   acdvd|         wts|                60|                    1|    1|            3|
| 253704|        1|         caval|           1837|    arbk|   